In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


# LeNet

In [ ]:
# 定义 LeNet 模型
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 定义卷积层
        self.features = nn.Sequential(
            # Conv1
            nn.Conv2d(1, 20, kernel_size=5),
            nn.ReLU(),
            # Pool1
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            # Conv2
            nn.Conv2d(20, 50, kernel_size=5),
            nn.ReLU(),
            # AdaptiveMaxPool2d
            nn.AdaptiveMaxPool2d(output_size=(4, 4)),
        )
        
        # 定义全连接层
        self.classify = nn.Sequential(
            # FC1
            nn.Linear(50 * 4 * 4, 500),
            nn.ReLU(),
            # FC2
            nn.Linear(500, 10),
        )

    def forward(self, x):
        # 通过卷积层
        x = self.features(x)
        # 展平为一维向量
        x = torch.flatten(x, 1)
        # 通过全连接层
        x = self.classify(x)
        return x


# 设定超参数

In [3]:
# 设定超参数
batch_size = 64
learning_rate = 0.01
num_epochs = 3

# 加载 MNIST 数据集

In [4]:
# 加载 MNIST 数据集
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./datas', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./datas', train=False, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# 创建模型实例

In [5]:
# 创建模型实例
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
lenet = LeNet().to(device)

# 定义损失函数和优化器

In [6]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(lenet.parameters(), lr=learning_rate)

# 训练模型

In [9]:
# 训练模型
for epoch in range(num_epochs):
    lenet.train()
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        # 前向传播
        outputs = lenet(images)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    
    # 测试模型
    lenet.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = lenet(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}%')

print('Finished Training')

Epoch [1/3], Step [100/938], Loss: 0.0869
Epoch [1/3], Step [200/938], Loss: 0.1285
Epoch [1/3], Step [300/938], Loss: 0.0606
Epoch [1/3], Step [400/938], Loss: 0.0996
Epoch [1/3], Step [500/938], Loss: 0.0632
Epoch [1/3], Step [600/938], Loss: 0.0998
Epoch [1/3], Step [700/938], Loss: 0.0297
Epoch [1/3], Step [800/938], Loss: 0.0456
Epoch [1/3], Step [900/938], Loss: 0.0626
Accuracy of the network on the 10000 test images: 98.22%
Epoch [2/3], Step [100/938], Loss: 0.0166
Epoch [2/3], Step [200/938], Loss: 0.0145
Epoch [2/3], Step [300/938], Loss: 0.0351
Epoch [2/3], Step [400/938], Loss: 0.0129
Epoch [2/3], Step [500/938], Loss: 0.0238
Epoch [2/3], Step [600/938], Loss: 0.0349
Epoch [2/3], Step [700/938], Loss: 0.0730
Epoch [2/3], Step [800/938], Loss: 0.1704
Epoch [2/3], Step [900/938], Loss: 0.0056
Accuracy of the network on the 10000 test images: 98.3%
Epoch [3/3], Step [100/938], Loss: 0.0355
Epoch [3/3], Step [200/938], Loss: 0.0428
Epoch [3/3], Step [300/938], Loss: 0.0497
Epoch

In [10]:
total, correct

(10000, 9860)